In [1]:
%load_ext autoreload
%autoreload 2
    
import copy
import os
import torch
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.config import CfgNode as CN
from detectron2.data import (
    MetadataCatalog,
    build_detection_test_loader,
    build_detection_train_loader,
)
from detectron2.engine import DefaultTrainer, DefaultPredictor, default_argument_parser, default_setup, launch
from detectron2.evaluation import inference_on_dataset
from detectron2.modeling import build_model
from detectron2.utils.logger import setup_logger

import scene_generation.inverse_graphics.synthetic_scene_database_loader as data
import scene_generation.inverse_graphics.roi_heads as roi_heads

DATA_ROOT = "/home/gizatt/data/generated_cardboard_envs/"
DETECTRON_ROOT = "/home/gizatt/tools/detectron2/"

%matplotlib inline
def cv2_imshow(im):
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
    plt.axis('off')

In [2]:
from detectron2.data import DatasetCatalog, MetadataCatalog
from scene_generation.inverse_graphics.synthetic_scene_database_loader import load_xencoco_json

DatasetCatalog.clear()
def load_dataset(d):
    return load_xencoco_json(
        os.path.join(DATA_ROOT, "%s.json" % (d)),
        data_root=DATA_ROOT,
        dataset_name="synthetic_%s" % d)
def load_real_dataset():
    return detectron2.data.datasets.load_coco_json(
        "/home/gizatt/data/coco/cardboard_boxes_in_wild/coco.json",
        image_root="/home/gizatt/data/coco/cardboard_boxes_in_wild/images",
        dataset_name="prime_boxes_real", extra_annotation_keys=None)
for d in ["train", "test"]:
    DatasetCatalog.register("synthetic_" + d, lambda d=d: load_dataset(d))
DatasetCatalog.register("prime_boxes_real", load_real_dataset)
synthetic_train_metadata = MetadataCatalog.get("synthetic_train")
real_prime_boxes_metadata = MetadataCatalog.get("prime_boxes_real")
real_prime_boxes_metadata.set(json_file="/home/gizatt/data/coco/cardboard_boxes_in_wild/coco.json")
print("Metadata train: ", synthetic_train_metadata)
print("Metadata real: ", real_prime_boxes_metadata)

Metadata train:  Metadata(name='synthetic_train')
Metadata real:  Metadata(json_file='/home/gizatt/data/coco/cardboard_boxes_in_wild/coco.json', name='prime_boxes_real')


In [3]:
cfg = get_cfg()
cfg.merge_from_file(os.path.join(DETECTRON_ROOT, "configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

cfg.INPUT.MIN_SIZE_TRAIN = (480)
cfg.DATALOADER.ASPECT_RATIO_GROUPING = False

cfg.DATASETS.TRAIN = ("synthetic_train",)
cfg.DATASETS.TEST = ("synthetic_test",)
cfg.DATALOADER.NUM_WORKERS = 0
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # default
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (prime box)
cfg.MODEL.ROI_HEADS.NAME = "XenRCNNROIHeads"

cfg.MODEL.ROI_SHARED_HEAD = CN()
cfg.MODEL.ROI_SHARED_HEAD.POOLER_RESOLUTION = 14
cfg.MODEL.ROI_SHARED_HEAD.POOLER_SAMPLING_RATIO = 2
cfg.MODEL.ROI_SHARED_HEAD.POOLER_TYPE = "ROIAlign"

cfg.MODEL.ROI_SHAPE_HEAD = CN()
cfg.MODEL.ROI_SHAPE_HEAD.NAME = "RCNNShapeHead"
cfg.MODEL.ROI_SHAPE_HEAD.NUM_CONV = 4
cfg.MODEL.ROI_SHAPE_HEAD.CONV_DIM = 3
cfg.MODEL.ROI_SHAPE_HEAD.NUM_FC = 2
cfg.MODEL.ROI_SHAPE_HEAD.FC_DIM = 100
cfg.MODEL.ROI_SHAPE_HEAD.NORM = ""
cfg.MODEL.ROI_SHAPE_HEAD.NUM_SHAPE_PARAMS = 3


#cfg.freeze()
train_loader = build_detection_test_loader(cfg, dataset_name="synthetic_test", mapper=data.XenRCNNMapper(cfg, True))

In [4]:
model = build_model(cfg)
model.eval()

/home/gizatt/miniconda3/envs/py36_pyro/lib/python3.6/site-packages/torch/cuda/__init__.py:134: UserWarning: 
    Found GPU1 GeForce GTX 680 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2): Sequential(
        (0): BottleneckBlock

In [8]:
training_example = next(iter(train_loader))
with torch.no_grad():
    print("Run: ", model(training_example))

Run:  [{'instances': Instances(num_instances=4, image_height=480, image_width=640, fields=[pred_boxes: Boxes(tensor([[  0.0000,   0.0000, 640.0000, 401.2548],
        [  0.0000, 147.8126, 640.0000, 255.8278],
        [  0.0000, 363.1194, 640.0000, 409.8233],
        [  0.0000, 132.1335, 640.0000, 134.7401]], device='cuda:0')), scores: tensor([1., 1., 1., 1.], device='cuda:0'), pred_classes: tensor([0, 0, 0, 0], device='cuda:0'), pred_shape_params: tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], device='cuda:0')])}]


In [7]:

class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, params=None):
        evaluator_type = MetadataCatalog.get(dataset_name).evaluator_type
        if evaluator_type == "pix3d":
            return Pix3DEvaluator(
                dataset_name,
                cfg,
                True,
                mesh_models=params["mesh_models"] if "mesh_models" in params else None,
            )g")

    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        return build_detection_test_loader(cfg, dataset_name, mapper=MeshRCNNMapper(cfg, False))

    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=MeshRCNNMapper(cfg, True))

    @classmethod
    def test(cls, cfg, model):
        """
        Args:
            cfg (CfgNode):
            model (nn.Module):
        Returns:
            dict: a dict of result metrics
        """
        results = OrderedDict()
        for dataset_name in cfg.DATASETS.TEST:
            data_loader = cls.build_test_loader(cfg, dataset_name)
            # ugly hack to use preloaded models to save time from loading them again
            vars = {"mesh_models": copy.deepcopy(data_loader.dataset._dataset[0]["mesh_models"])}
            evaluator = cls.build_evaluator(cfg, dataset_name, params=vars)
            results_i = inference_on_dataset(model, data_loader, evaluator)
            results[dataset_name] = results_i
            if comm.is_main_process():
                assert isinstance(
                    results_i, dict
                ), "Evaluator must return a dict on the main process. Got {} instead.".format(
                    results_i
                )
        return results

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 41)